In [1]:
import numpy as np
from typing import Tuple, Generator, Iterator

# for plotting voxels
import matplotlib.pyplot as plt
from functools import reduce
from mpl_toolkits.mplot3d import Axes3D # registers the 3D projection, unused

In [17]:
len(set([1,2,3]))

3

In [23]:
class ColumnBunches3():
    
    def __init__(self, data: np.ndarray, window: int, skip_diag=False, diag_only=False):
        
        self.data = data
        self.N = data.shape[1]
        self.w = window
        self.sd = int(skip_diag)
        
        self.M = (self.N-1) // self.w + 1
        
        # I, J, K indeces identify bunches of w columns
        # they generally sit in range(0:M)
        self.IJK = self._IJK_gen() if not diag_only else self._IJK_diag_gen()
        
    def _IJK_gen(self) -> Generator:
        for I in range(self.M):
            for J in range(I + self.sd, self.M):
                for K in range(J + self.sd, self.M):
                    yield I, J, K
                
    def _IJK_diag_gen(self) -> Generator:
        for I in range(self.M):
            for J in range(I, self.M):
                for K in range(J, self.M):
                    if len({I, J, K}) < 3:
                        yield I, J, K
    
    def _get_columns(self, I: int, J: int, K: int) -> Tuple:
        return self.data[:, self.w*I : self.w*(I+1)], \
               self.data[:, self.w*J : self.w*(J+1)], \
               self.data[:, self.w*K : self.w*(K+1)]
    
    def __next__(self) -> Tuple:
        return self._get_columns(*next(self.IJK))
    
    def __iter__(self) -> Iterator:
        return self
    
    def voxels(self, invertY = True) -> None:
        fig = plt.figure()
        ax = fig.gca(projection='3d')
        plt.cla()
        ax.grid(False)
        ax.set_yticks(range(self.M))
        ax.set_xticks(range(self.M))
        ax.set_zticks(range(self.M))
        ax.set_ylabel('I')
        ax.set_xlabel('J')
        ax.set_zlabel('K')
        if invertY: ax.invert_yaxis()
        x, y, z = np.indices((self.M,)*3)
        masks = ( (y == I) & (x == J) & (z == K) for I, J, K in self._IJK_gen() )
        voxels = reduce(lambda m1,m2: m1|m2, masks, np.empty((self.M,)*3, dtype=bool))
        ax.voxels(voxels, edgecolor='k')
        plt.show()

In [19]:
M1 = np.array([[1.,2,3,4,5]]*3)
M1

array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.]])

In [14]:
bunches = ColumnBunches3(M1, window=2)
list(bunches._IJK_gen())

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 2),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 2),
 (2, 2, 2)]

In [15]:
%matplotlib qt
bunches.voxels(invertY=False)

In [24]:
bunches = ColumnBunches3(M1, window=3)

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------


In [25]:
bunches = ColumnBunches3(M1, window=3, skip_diag=True) # None

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

In [26]:
bunches = ColumnBunches3(M1, window=3, diag_only=True)

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------


In [31]:
[2,3,1].sort()